### Loading Airbnb data into database

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc

pd.set_option('display.max_columns', None)

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-QEE21EV;'
                      'Database=Airbnb_db;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [3]:
airbnb = pd.read_csv('airbnb_data.csv', delimiter=";")

In [5]:
airbnb.head()

,id,scrape_id,last_scraped,name,host_id,host_name,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,property_type,room_type,accommodates,bathrooms_type,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,city,region,country,bathrooms
0,18674,20210412154247,2021-04-13,Huge flat for 8 people close to Sagrada Familia,71615,Mireia And Maria,2010-01-19,within a day,50%,80%,f,34,t,t,41.40556,2.17262,Entire apartment,Entire home/apt,8,private,3,6,"[""Air conditioning"", ""Host greets you"", ""Essen...",157.0,1,1125,27,21,2013-05-27,2019-11-01,88,9,10,10,10,10,9,t,0.22,Barcelona,Catalonia,Spain,2.0
1,23197,20210412154247,2021-04-14,Forum CCIB DeLuxe★Spacious &Elegant ★Large Bal...,90417,Etain (Marnie),2010-03-09,within an hour,100%,78%,f,2,t,t,41.41291,2.22063,Entire apartment,Entire home/apt,6,private,3,5,"[""High chair"", ""Washer"", ""Refrigerator"", ""Sham...",220.0,3,300,359,52,2011-03-15,2019-12-15,95,10,10,10,10,9,9,f,0.42,Barcelona,Catalonia,Spain,2.0
2,32711,20210412154247,2021-04-14,Sagrada Familia area - Còrsega 1,135703,Nick,2010-05-31,within an hour,100%,100%,f,5,t,t,41.40566,2.17015,Entire apartment,Entire home/apt,6,private,2,3,"[""High chair"", ""Washer"", ""Refrigerator"", ""Sham...",144.0,2,31,0,63,2011-07-17,2020-02-22,86,9,9,10,10,10,9,t,0.53,Barcelona,Catalonia,Spain,1.5
3,34241,20210412154247,2021-04-15,Private Double room - Plaza Real,73163,Andres,2010-01-24,within an hour,90%,70%,f,3,t,t,41.38062,2.17517,Private room in apartment,Private room,2,private,1,1,"[""Dishwasher"", ""Air conditioning"", ""Oven"", ""Wi...",50.0,30,240,0,8,2010-07-10,2013-07-15,68,8,8,7,9,8,7,f,0.06,Barcelona,Catalonia,Spain,1.0
4,34981,20210412154247,2021-04-15,VIDRE HOME PLAZA REAL on LAS RAMBLAS,73163,Andres,2010-01-24,within an hour,90%,70%,f,3,t,t,41.37978,2.17623,Entire apartment,Entire home/apt,9,private,4,6,"[""High chair"", ""Washer"", ""Refrigerator"", ""Sham...",243.0,4,365,114,149,2010-10-03,2020-03-11,91,10,9,10,10,10,9,f,1.16,Barcelona,Catalonia,Spain,3.0


In [6]:
airbnb.shape

(188437, 43)

In [7]:
total = airbnb.isna().sum().sort_values(ascending=False) # how many missings in each column
percent = (airbnb.isna().sum()/airbnb.isna().count()).round(4).sort_values(ascending=False) # in %
missing_df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_df.query("Total > 0")

,Total,Percent
host_response_time,60957,0.3235
host_response_rate,60957,0.3235
host_acceptance_rate,43685,0.2318


In [3]:
cursor.execute('CREATE TABLE Listings ( \
               id int, \
               scrape_id float, \
               last_scraped  date, \
               name nvarchar(500), \
               host_id int, \
               host_name nvarchar(50), \
               host_since  date, \
               host_response_time nvarchar(20), \
               host_response_rate nvarchar(5), \
               host_acceptance_rate nvarchar(5), \
               host_is_superhost nvarchar(2), \
               host_listings_count  int, \
               host_has_profile_pic nvarchar(2), \
               host_identity_verified  nvarchar(2), \
               latitude nvarchar(50), \
               longitude nvarchar(50), \
               property_type nvarchar(50), \
               room_type  nvarchar(50), \
               accommodates int , \
               bathrooms_type nvarchar(10), \
               bedrooms int, \
               beds int, \
               amenities nvarchar(2500), \
               price nvarchar(10), \
               minimum_nights int, \
               maximum_nights int, \
               availability_365  int, \
               number_of_reviews  int, \
               first_review date, \
               last_review date, \
               review_scores_rating int, \
               review_scores_accuracy int, \
               review_scores_cleanliness int, \
               review_scores_checkin int, \
               review_scores_communication int, \
               review_scores_location int, \
               review_scores_value int, \
               instant_bookable  nvarchar(2), \
               reviews_per_month nvarchar(5), \
               city nvarchar(50), \
               region nvarchar(50), \
               country nvarchar(50), \
               bathrooms nvarchar(5) )')
conn.commit()

In [1]:
# for row in airbnb.itertuples():
#     cursor.execute('''
#                 INSERT INTO Listings2
#                 VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?, \
#                 ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
#                 ''',
#                row.id,
#                row.scrape_id,
#                row.last_scraped,
#                row.name,
#                row.host_id,
#                row.host_name,
#                row.host_since,
#                row.host_response_time,
#                row.host_response_rate,
#                row.host_acceptance_rate,
#                row.host_is_superhost,
#                row.host_listings_count,
#                row.host_has_profile_pic,
#                row.host_identity_verified,
#                row.latitude,
#                row.longitude,
#                row.property_type,
#                row.room_type,
#                row.accommodates,
#                row.bathrooms_type,
#                row.bedrooms,
#                row.beds,
#                row.amenities,
#                row.price,
#                row.minimum_nights,
#                row.maximum_nights,
#                row.availability_365,
#                row.number_of_reviews,
#                row.first_review,
#                row.last_review,
#                row.review_scores_rating,
#                row.review_scores_accuracy,
#                row.review_scores_cleanliness,
#                row.review_scores_checkin,
#                row.review_scores_communication,
#                row.review_scores_location,
#                row.review_scores_value,
#                row.instant_bookable,
#                row.reviews_per_month,
#                row.city,
#                row.region,
#                row.country,
#                row.bathrooms)
# conn.commit()
